In [1]:
import os
os.chdir('../..')

In [2]:
from sdv.evaluation import evaluate

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

import pickle

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Functions

In [3]:
def is_a_DATGAN(name):
    if any(x in name for x in ['TGAN', 'CTGAN', 'FULL', 'TRANSRED', 'LINEAR', 'NOLINKS', 'PREDICTION']):
        return False
    else:
        return True
    
def check_low_appearing_vars(df):
    
    for c in df.columns:
        val = df[c].value_counts()
        if len(val) < 20:
            val = val/len(df)
            if any(val < 0.01) and c != 'choice':
                print('Variable {}: '.format(c))
                for idx, v in zip(val.index, val):
                    if v < 0.01:
                        print('  {} - {:.2f}% ({:d})'.format(idx, 100*v, int(v*len(df))))
                print()
                
def replace_low_appearing_values(df, dataset):
    
    if 'Chicago' in dataset:
        dct_ = {}
        for i in df['hh_vehicles'].unique():
            if i >= 5:
                dct_[i] = '5+'
            else:
                dct_[i] = str(i)        
        df['hh_vehicles'].replace(dct_, inplace=True)
        
        dct_ = {}
        for i in df['hh_size'].unique():
            if i >= 6:
                dct_[i] = '6+'
            else:
                dct_[i] = str(i)        
        df['hh_size'].replace(dct_, inplace=True)
        
        dct_ = {}
        for i in df['hh_bikes'].unique():
            if i >= 6:
                dct_[i] = '6+'
            else:
                dct_[i] = str(i)        
        df['hh_bikes'].replace(dct_, inplace=True)       

    elif 'LPMC' in dataset:
        dct_ = {}
        for i in df['pt_n_interchanges'].unique():
            if i >= 2:
                dct_[i] = '2+'
            else:
                dct_[i] = str(i)        
        df['pt_n_interchanges'].replace(dct_, inplace=True) 
        
        dct_ = {
            'Diesel_LGV': 'LGV',
            'Petrol_LGV': 'LGV',
            'Hybrid_Car': 'Average_Car'
        }

In [4]:
dataset = 'Chicago'
orig_str = 'random-original'
input_folder = '../synth_data/{}/'.format(dataset)
n_models = 5
n_data = 5

# Models for testing all DATGANS
models = ['CTGAN', 'TGAN']

for i in ['WGAN', 'SGAN', 'WGGP']:
    for j in ['WI', 'OR', 'WO']:
        for k in ['NO', 'BO', 'OD']:
            models.append('{}_{}_{}'.format(i,j,k))
            
# Models for testing different DAGs
if 'DAG' in dataset:
    models = ['FULL', 'TRANSRED', 'LINEAR', 'NOLINKS', 'PREDICTION']

models.sort()

files_ = {}

for m in models:
    tmp = []
    if is_a_DATGAN(m):
        spl = m.split('_')
        for i in range(n_models):
            for j in range(n_data):
                tmp.append(input_folder + '{}_{}_{:0>2}_{}_{:0>2}.csv'.format(spl[0], spl[1], i+1,  spl[2], j+1))
    else:
        for i in range(n_models):
            for j in range(n_data):
                tmp.append(input_folder + '{}_{:0>2}_{:0>2}.csv'.format(m, i+1, j+1))
    files_[m] = tmp

In [5]:
if 'Chicago' in dataset:
    continuous_columns = ["distance", "age", "departure_time"]
elif 'LPMC' in dataset:
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [6]:
df_orig = pd.read_csv('../data/' + dataset.split('_')[0] + '/data.csv')
replace_low_appearing_values(df_orig, dataset)
check_low_appearing_vars(df_orig)

In [7]:
cat_cols = list(set(df_orig.columns) - set(continuous_columns))

In [8]:
filepath = './notebooks/results/{}/'.format(dataset)
filename = 'sdv_results.pickle'
sdv_results = {}
n = 7000

if 'Chicago' in dataset:
    target_col = 'choice'
elif 'LPMC' in dataset:
    target_col = 'mode_choice'
    
try:
    sdv_results = pickle.load(open(f'{filepath}{filename}','rb'))
    print('Found previous picle file, using that')
except:
    print('No previous results found, starting fresh')
    try:
        os.makedirs(filepath)
    except:
        pass

Found previous picle file, using that


In [ ]:
for i, model in enumerate(models):
    
    if model in sdv_results.keys():
        print("Results for model \033[1m{}\033[0m ({}/{}) already exists".format(model, i+1, len(models)))
    else:
        print("Getting results for model \033[1m{}\033[0m ({}/{})".format(model, i+1, len(models)))
        sdv_results[model] = []
        
    n_files_done = len(sdv_results[model])
    
    for j, f in enumerate(files_[model][n_files_done:]):
        print("  Processing file {}/{}".format(j+1+n_files_done, len(files_[model])) + " "*20)

        tmp_df = pd.read_csv(f)
        replace_low_appearing_values(tmp_df, dataset)
        
        res = evaluate(tmp_df, df_orig, aggregate=False)
        
        tmp = {}
        for i in res.index:
            row = res.loc[i]
            tmp[row.metric] = row.normalized_score

        sdv_results[model].append(tmp)
        
        pickle.dump(sdv_results, open(filepath + filename, 'wb'))

Results for model CTGAN (1/29) already exists
  Processing file 6/25                    


D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sdmetrics\column_pairs\statistical\kl_divergence.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data[pd.isnull(real_data)] = 0.0
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\pandas\core\frame.py:3089: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sdmetrics\column_pairs\statistical\kl_divergence.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

  Processing file 7/25                    


D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sdmetrics\column_pairs\statistical\kl_divergence.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data[pd.isnull(real_data)] = 0.0
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\pandas\core\frame.py:3089: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sdmetrics\column_pairs\statistical\kl_divergence.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

  Processing file 8/25                    


...

  Processing file 8/25                    


D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sdmetrics\column_pairs\statistical\kl_divergence.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data[pd.isnull(real_data)] = 0.0
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\pandas\core\frame.py:3089: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sdmetrics\column_pairs\statistical\kl_divergence.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

  Processing file 9/25                    


D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sdmetrics\column_pairs\statistical\kl_divergence.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data[pd.isnull(real_data)] = 0.0
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\pandas\core\frame.py:3089: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
D:\Users\glede\anaconda3\envs\py37\lib\site-packages\sdmetrics\column_pairs\statistical\kl_divergence.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

  Processing file 10/25                    


In [10]:
res

,metric,name,raw_score,normalized_score,min_value,max_value,goal
0,BNLogLikelihood,BayesianNetwork Log Likelihood,-8.959152,0.000129,-inf,0.0,MAXIMIZE
1,LogisticDetection,LogisticRegression Detection,0.629658,0.629658,0.0,1.0,MAXIMIZE
2,SVCDetection,SVC Detection,0.440365,0.440365,0.0,1.0,MAXIMIZE
11,GMLogLikelihood,GaussianMixture Log Likelihood,-5.353331,0.004710,-inf,inf,MAXIMIZE
12,CSTest,Chi-Squared,0.998850,0.998850,0.0,1.0,MAXIMIZE
13,KSTest,Inverted Kolmogorov-Smirnov D statistic,0.913792,0.913792,0.0,1.0,MAXIMIZE
14,KSTestExtended,Inverted Kolmogorov-Smirnov D statistic,0.925449,0.925449,0.0,1.0,MAXIMIZE
27,ContinuousKLDivergence,Continuous Kullback–Leibler Divergence,0.906410,0.906410,0.0,1.0,MAXIMIZE
28,DiscreteKLDivergence,Discrete Kullback–Leibler Divergence,0.939871,0.939871,0.0,1.0,MAXIMIZE


In [14]:
tmp = {}
for i in res.index:
    row = res.loc[i]
    tmp[row.metric] = row.normalized_score